# Theory of Finance

In this notebook, you will work with stock market data from the S&P500. You will learn some basics of portfolio management using R, such as computing log-returns, computing and plotting the mean-variance frontier, and building the minimum variance portfolio.

In [ ]:
# Library to access Yahoo!Finance data
install.packages("quantmod") # The quantmod library is not in the default libraries, it must be installed first
library(quantmod)
# Time series library
library(xts)
# Plotting library
library(ggplot2);

In [2]:
# Set plot size for upcoming plots
options(repr.plot.width = 12, repr.plot.height = 8)

## Obtaining financial data

The first thing we are going to do is gather stock prices from the `quantmod` library we have just loaded above. In general, there exist a lot of different sources for financial data, depending on your requirements (which markets, which frequency, which time window, etc.) you might need to look into further sources. For simplicity's sake, this notebook will ignore these complications but these are things you should consider when writing your research.


We begin by building a list of 30 *random* tickers from stocks in the S&P500. We will grab the data from `quantmod` for these stocks only.


In [3]:
# Create a list with the tickers of the stocks for which we want to obtain data
stocklist <- c('AAPL', 'AFL', 'AMZN', 'APD', 'BAC', 'COF', 'COST', 
             'HBAN', 'HON', 'INTU', 'IPG', 'KEY', 'KLAC', 'LMT', 
             'LOW', 'MSFT', 'NEM', 'NKE', 'OMC', 'PAYX', 'RF', 'RHI', 
             'SNA', 'SWK', 'UPS', 'USB', 'VZ', 'WHR', 'WY', 'YUM')

Because we are querying an online API, obtaining the data can take a few seconds.

In [ ]:
# Obtain the data from Yahoo!Finance between 1st Jan 2010 and 1st Jun 2021
getSymbols(stocklist, src = "yahoo", from = as.Date("2010-01-01"), to = as.Date("2021-06-01"))
# Keep only "Adjusted Close" prices and build a time series objects
prices <- xts(data.frame(matrix(unlist(lapply(stocklist, function(x) Ad(get(x)))), nrow = length(Ad(get(stocklist[1]))), byrow=FALSE)), order.by = index(get(stocklist[1])))
names(prices) <- stocklist
# Display the first 10 rows of the dataset
head(prices, 10)

In [ ]:
# Create a dataframe of percentage returns
returns <- 100 * (exp(diff(log(prices), 1)) - 1)
# Display the first 10 rows
head(returns, 10)

In [6]:
# Since the first row is NA, we drop it to avoid dealing with NAs
returns <- returns[2:nrow(returns), ]

Now that we have obtained stock returns for our 30 stocks between 2010 and 2021, we further proceed by splitting the data into train and test sets. In general, a train set is a set you use to fit your model and the test set is used to validate your model. This can help us understand when a model is overfitting. An overfitted model will perform well on the train data (the data that was used to fit the model) and it will perform badly on the test set (data that the model has not "*seen*" yet).

In [ ]:
# Separate the data into a training (pre-2020) and a testing set (post-2019)
train_index <- index(returns) < "2020-1-1"
test_index <- index(returns) >= "2020-1-1"
# Display the last 10 rows of the training set
tail(returns[train_index], 10)

In [8]:
# Using the training set, compute the stock returns means,
mean_returns <- colMeans(returns[train_index])
# ... standard deviations,
std_returns <- apply(returns[train_index], 2, sd)
# ... and covariances
cov_returns <- cov(returns[train_index])

## Computing the efficient frontier

The efficient frontier (also mean-variance or portfolio frontier) is a set of portfolio allocations which offer the lowest variance for a given level of return.

Finding the efficient frontier is done by solving the optimization problem

$$
\begin{align}
\min &\mathbb{V}[R_p] \\ \text{s.t.} \ &\mathbb{E}[R_p] = \mu^\star
\end{align}
$$
where $\mathbb{V}[R_p]$ represents the variance of the portfolio and $\mathbb{E}[R_p]$ its expected return. Additionally, it is imperative that the chosen portfolio sum to 1.

The efficient frontier can be obtained through numerical minimization or by linear algebra, we follow the latter approach, as given in https://github.com/PaulSoderlind/FinancialTheoryMSc/blob/master/Ch03_MeanVariance.ipynb (however, the linked notebook implements the approach in Julia, here it is of course done in Python)

In [9]:
# Compute the efficient frontier
I <- rep(1, length(mean_returns)) # Vector of ones
S_1 <- solve(cov_returns)         # Inverse of the return covariance matrix
# Compute scalars to help us with the computation of the frontier 
# # (see course slides, chapter 3.1.4)
A <- c(t(mean_returns) %*% S_1 %*% mean_returns)
B <- c(t(mean_returns) %*% S_1 %*% I)
C <- c(t(I) %*% S_1 %*% I)
# Create a function that returns the weights that minimize the variance for a 
# given level of returns mu_star
mv_frontier <- function(mu_star) {
  lam <- (C * mu_star - B) / (A * C - B^2)
  delta <- (A - B * mu_star) / (A * C - B^2)
  w <- S_1 %*% (mean_returns * lam + I * delta)
  list(mu = c(sqrt(t(w) %*% cov_returns %*% w)), w = w)
}

In [10]:
# Create the range of the y-axis
mu_star <- seq(from = 0, to = 0.2, by = 0.001)
mvf <- sapply(mu_star, mv_frontier)[1, ] # Compute the efficient frontier

In [11]:
# Compute the limits for the x-axis
limx <- c(min(unlist(mvf)) * 0.9, max(std_returns) * 1.1)

In [ ]:
# Plot the mean-variance efficient frontieras a red line
plot(sapply(mu_star, mv_frontier)[1, ], mu_star, xlab = "Standard deviation",
  ylab = "Mean", main = "Returns' mean-stdev plot", type = "l", xlim = limx,
  ylim = c(min(mu_star), max(mu_star)), col = "red")
# Plot individual stock mean returns and std as blue dots
points(mean_returns ~ std_returns, col = "blue")
text(mean_returns ~ std_returns, labels = stocklist, pos = 3, col = "blue")

## Comparing portfolios

We now compute the weights that produce the minimum variance portfolio on the train set. Using the equal weights portfolio as a benchmark, we compare the performance of both portfolios on the test set.

In [15]:
# Compute the min-variance portfolio weights
w_minvar <- as.vector(S_1 %*% I) / as.numeric(t(I) %*% S_1 %*% I)
# Store uniform weights for a benchmark portfolio
w_uniform <- rep(1, ncol(prices)) / ncol(prices)

In [16]:
# Create a function to return an array of portfolio value over time
portfolio_value <- function(prices, weights) {
  # Compute the logarithmic returns
  log_returns <- diff(log(prices), 1)
  # Store a vector of dates to return a timeseries object later
  dates <- as.Date(index(log_returns[complete.cases(log_returns)]), "%Y-%m-%d")
  # Multiply the log returns by the associated weights
  log_returns <- log_returns %*% weights
  # Take the cumulative sum where the result is not NA and exponentiate 
  # to transform back to percentage returns
  xts(x = exp(cumsum(log_returns[!is.na(log_returns)])), order.by = dates)
}

In [40]:
# Compute the returns for both portfolios for the train and test set
r_minvar_train <- portfolio_value(prices[train_index], w_minvar)
r_uniform_train <- portfolio_value(prices[train_index], w_uniform)
r_minvar_test <- portfolio_value(prices[test_index], w_minvar)
r_uniform_test <- portfolio_value(prices[test_index], w_uniform)
# Merge train and test results in separate xts objects
r_train <- merge(r_minvar_train, r_uniform_train)
r_test <- merge(r_minvar_test, r_uniform_test)

In [ ]:
plt <- plot.xts(r_train, major.ticks = "years", main = "Train set", 
  yaxis.right = FALSE)
addLegend("topleft", legend.names = c("Min-variance", "Equal weights"), lty=1, 
  lwd=2)

In [ ]:
plt <- plot.xts(r_test, major.ticks = "years", main = "Train set", 
  yaxis.right = FALSE)
addLegend("topleft", legend.names = c("Min-variance", "Equal weights"), lty=1, 
  lwd=2)

We see that, for both the train and test data, the equally weighted portfolio outperforms the minimum-variance portfolio returns-wise, however, it is also quite obvious that its variance is much higher.

To get a better grasp of these differences, we compute the mean, standard deviation, and Sharpe ratio of both portfolios for both the train and test period.

Because computing means of percentage returns does not make sense, we will instead work with log-returns for the portfolio metrics.

In [35]:
# Create a function to output portfolio metrics
portfolio_metrics <- function(prices, weights) {
  # Compute the logarithmic returns
  log_returns <- diff(log(prices), 1)
  # Compute the log-returns for the portfolio
  log_returns <- log_returns %*% weights
  # Compute and display the portfolio metrics
  mu <- mean(log_returns, na.rm = TRUE)
  sigma <- sd(log_returns, na.rm = TRUE)
  print(paste0("Mean:    ", round(mu, digits = 5)))
  print(paste0("St. dev.:", round(sigma, digits = 5)))
  print(paste0("Sharpe:  ", round(mu / sigma, digits = 5)))
}

In [ ]:
print("Min. variance portfolio (train data)")
print("-----------------------------------------")
portfolio_metrics(prices[train_index], w_minvar)
print("") # Empty line
print("Equally weighted portfolio (train data)")
print("-----------------------------------------")
portfolio_metrics(prices[train_index], w_uniform)

In [ ]:
print("Min. variance portfolio (test data)")
print("-----------------------------------------")
portfolio_metrics(prices[test_index], w_minvar)
print("") # Empty line
print("Equally weighted portfolio (test data)")
print("-----------------------------------------")
portfolio_metrics(prices[test_index], w_uniform)

Given that we are working with log-returns, we can simply annualize the daily performance measures by multiplying with 252 (returns) and square-root of 252 (for standard deviation and Sharpe-ratio).

In [38]:
# Create a function to output portfolio metrics
portfolio_metrics_an <- function(prices, weights) {
  # Compute the logarithmic returns
  log_returns <- diff(log(prices), 1)
  # Compute the log-returns for the portfolio
  log_returns <- log_returns %*% weights
  # Compute and display the portfolio metrics
  mu <- mean(log_returns, na.rm = TRUE)
  sigma <- sd(log_returns, na.rm = TRUE)
  print(paste0("Mean:    ", round(mu * 252, digits = 5)))
  print(paste0("St. dev.:", round(sigma * sqrt(252), digits = 5)))
  print(paste0("Sharpe:  ", round(mu * sqrt(252) / sigma, digits = 5)))
}

In [ ]:
print("Min. variance portfolio (test data, annualized)")
print("-----------------------------------------")
portfolio_metrics_an(prices[test_index], w_minvar)
print("") # Empty line
print("Equally weighted portfolio (test data, annualized)")
print("-----------------------------------------")
portfolio_metrics_an(prices[test_index], w_uniform)